## Subcellular Timelapse seq: TC conversion rate estimation
Author: Robert Ietswaart  
Date: 20210625  
License: BSD2.  
Load modules j3dl and activate virtual environment using j4RNAdecay on O2.  
Python v3.7.4, pytorch v1.4 

source: `TCconversion_from_background_20210506.ipynb`  
For Brendan's project: estimate with binomial mixture model the TC conversion rates from the full no4sU and treated sample distributions of human (K562) experiments `T` and `U` for all available fractions.

In [1]:
import os
import pandas as pd
import numpy as np
import math
import copy
import matplotlib.pyplot as plt
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
import seaborn as sns
from scipy.optimize import least_squares
from lmfit import minimize, Parameters
import new_total_ratio as ntr
import fit

In [44]:
path = os.path.join('/n', 'groups', 'churchman', 'bms36', 
                    'TimeLapse_for_Robert', 'binom_method', 'human')
outpath = os.path.join('/n','groups','churchman','ri23','bseq','GS20210625_human')
reps = ['T','U']
background_id = {r: '1' for r in reps}
time_id = [str(i) for i in range(1,6)]
fracs = ['chr','nuc', 'cyto', 'tot', 'poly']
time_mins = [0, 15, 30, 60, 120]
time_measured = pd.Series(time_mins[1:])


analysis_type = 'bottom500genes_turnover'
#'bottom500genes_turnover'
#'top1000genes_turnover'

RAW = dict()
A = dict()
for r in reps:
    for fr in fracs:
        for t in time_id:
            filename = r + t + '-' + fr + '_'+analysis_type+'_TcountANDTCconv.txt'
                
            if os.path.exists(os.path.join(path, filename)):
                RAW[r+fr+t]= pd.read_csv(os.path.join(path, filename), sep='\t', index_col=0)

                #creating A[n,k] matrix: insert zeros where RAW has not missing lines/columns
                N = RAW[r+fr+t].index[-1] + 1
                K = int(RAW[r+fr+t].columns[-1]) + 1
                A[r+fr+t] = np.zeros((N,K))
                for n in RAW[r+fr+t].index:
                    for k in RAW[r+fr+t].columns:
                        A[r+fr+t][n, int(k)] = RAW[r+fr+t][k][n]

### Fit background error rates

In [1]:
out = dict()
for r in reps:
    for fr in fracs:
        if r+fr+background_id[r] in A.keys():
            #Fit 1 error rates background
            params1 = Parameters()
            params1.add('Error_1', value=0.0001, min=0, max=1)
            out[r+fr+'err1'] = minimize(fit.calc_res_matrix, params1, method='leastsquare',
                                     args=(ntr.one_error_background, A[r+fr+background_id[r]], None))

            #Fit 2 error rates background
            params2 = Parameters()
            params2.add('Error_1', value=0.0001, min=0, max=1)
            params2.add('Error_2', value=0.00001, min=0, max=1)
            params2.add('frac_err', value=0.5, min=0, max=1)
            out[r+fr+'err2'] = minimize(fit.calc_res_matrix, params2, method='leastsquare',
                                        args=(ntr.two_error_background, A[r+fr+background_id[r]], None))

            #Confirm with AIC that 2 error model better explains the data
            prob1, prob2 = fit.calc_aic(out[r+fr+'err1'].ndata,
                                        out[r+fr+'err1'].chisqr,
                                        out[r+fr+'err2'].chisqr,
                                        out[r+fr+'err1'].nvarys,
                                        out[r+fr+'err2'].nvarys)
            print(r, fr, 'probability that model1 is favored', prob1)
            print(r, fr, 'probability that model2 is favored', prob2)

### Fit pc the TC conversion rate on treated samples

In [2]:
TC_INITS = [0.001,0.01]
FRAC_INITS = [0.05, 0.5]

paramsTC = Parameters()
for r in reps:
    for fr in fracs:
        print(r,fr)
        for t in time_id:
            print(t)
            if (not (t == background_id[r])) and (r+fr+t in A.keys()): 
                    chi2 = -1
                    for tc_init in TC_INITS:
                        for frac_init in FRAC_INITS:
                            paramsTC.add('TC', value=tc_init, min=0, max=1)
                            paramsTC.add('frac', value=frac_init, min=0, max=1)
                            out_temp = minimize(fit.calc_res_matrix, paramsTC, #method='leastsq',
                                        args=(ntr.TC_rate_from_background, A[r+fr+t], out[r+fr+'err2'].params))
                            if (chi2 < 0) or (chi2 > out_temp.chisqr):
                                out[r+fr+t] = out_temp
                                chi2 = out_temp.chisqr

# Write the predicted rates to file

In [3]:
for r in reps:
    for fr in fracs:
        if r+fr+background_id[r] in A.keys():
            output = pd.DataFrame()
            output['time'] = time_measured
            output_params = ['TC', 'frac']
            for para in output_params:
                output[para] = [out[r+fr+t].params[para].value for t in time_id[1:]]
                output[para+'_stderr'] = [out[r+fr+t].params[para].stderr for t in time_id[1:]]
            filename = r+'-'+fr+'_'+analysis_type+'_BMmodel_TC_rate_predictions' 
            output.to_csv(os.path.join(outpath, filename + '.tsv'), sep='\t', index=False)
            #also write full output: for completeness
            with open(os.path.join(outpath, filename + '_full.json'), 'w') as outfile:
                out[r+fr+t].params.dump(outfile)
            print(r, fr, '\n', output.head())
            
            output = pd.DataFrame()
            output['time'] = [0]
            output_params = ['Error_1', 'Error_2', 'frac_err']
            for para in output_params:
                output[para] = [out[r+fr+'err2'].params[para].value]
                output[para+'_stderr'] = [out[r+fr+'err2'].params[para].stderr]
            filename = r+'-'+fr+'_'+analysis_type+'_BMmodel_background_rate_predictions'
            output.to_csv(os.path.join(outpath, filename + '.tsv'), sep='\t', index=False)
            #also write full output: for completeness
            with open(os.path.join(outpath, filename + '_full.json'), 'w') as outfile:
                out[r+fr+'err2'].params.dump(outfile)
            print('\n', output.head(), '\n')
            
        else: 
            print('no need to export rates for', r, fr,' since no input files')